# Partial Labeled Data Preparing

## Set up

In [1]:
import os
from pathlib import Path
import shutil
import pandas as pd
from tqdm import tqdm
import time

import sys
PATH = Path.cwd().parents[1]
sys.path.append(str(PATH))

from backend.utils.data_loader import (
  get_info_by_contract_addr,
  save_bytecode_by_contract_addr,
  get_bytecode_by_contract_addr,
  save_transactions_by_contract_addr,
  get_source_code_by_contract_addr,
  save_sol_by_contract_addr
)

In [2]:
name = "crp_large_sample.csv" # change this
DATA_PATH = os.path.join(PATH, 'data')
SOURCE_PATH = os.path.join(DATA_PATH, 'external/crpwarner/dataset/large/hex') # change this
DATASET_PATH = os.path.join(DATA_PATH, 'interim')
UNLABELED_PATH = os.path.join(DATA_PATH, 'unlabeled')

## Move Dataset to Unlabeled

In [3]:
source = os.path.join(DATASET_PATH, name)
target = os.path.join(UNLABELED_PATH, name)
if os.path.exists(source):
    shutil.move(source, UNLABELED_PATH)
    print("File moved successfully.")
else:
    print("File does not exist.", DATASET_PATH)

File moved successfully.


In [4]:
df = pd.read_csv(target, index_col=0)
df.head()

,Mint,Leak,Limit
Address,,,
0x0290ea3c728981725689187763f6c63a68e192b8,-1,1,-1
0x02d3aea48b443a0026ed9cbc91b97d7335aba323,-1,-1,1
0x03260e1b0f53e1a1f93cf126a7ca42a1c71648d6,1,-1,-1
0x054ad3cd4a66f14bf5c0de2548a53be66995a4f6,-1,1,-1
0x0566c17dc2a9efcaa2f63e04cf06a69e8fc77f60,-1,0,-1


In [5]:
target_df = df.loc[df['Limit']!=-1] # change this
target_df.to_csv(os.path.join(UNLABELED_PATH, 'groundtruth.csv'))

In [6]:
HEX_PATH = os.path.join(UNLABELED_PATH, 'hex')
SOL_PATH = os.path.join(UNLABELED_PATH, 'sol')
TXN_PATH = os.path.join(UNLABELED_PATH, 'txn')
TMP_PATH = os.path.join(DATASET_PATH, 'hex')

In [7]:
for src_file in tqdm(list(Path(TMP_PATH).glob('*.hex'))):
    address = src_file.stem.lower()
    if address in target_df.index:
        shutil.move(src_file, os.path.join(HEX_PATH, f'{address}.hex'))

for src_file in tqdm(list(Path(SOURCE_PATH).glob('*.hex'))):
    address = src_file.stem.lower()
    if address in target_df.index:
        shutil.copy(src_file, os.path.join(HEX_PATH, f'{address}.hex'))
    else:
        shutil.copy(src_file, os.path.join(TMP_PATH, f'{address}.hex'))

100%|██████████| 13483/13483 [00:22<00:00, 607.00it/s]


In [8]:
len(target_df), len(os.listdir(HEX_PATH))

(91, 91)

In [9]:
chains = [1, 97, 56, 57054, 146]
# 3. Collect and extract
for addr in tqdm(target_df.index):
    print(f"Searching {addr}...")
    for chain in chains:
        print(f"Searching in {chain} chain...")
        if addr not in [filename.lower().split('.')[0] for filename in os.listdir(TXN_PATH)]:
            info = get_info_by_contract_addr(addr, chain)
            save_transactions_by_contract_addr(TXN_PATH, addr, info)
            if 'creationBytecode' in info.get('creator'):
                save_bytecode_by_contract_addr(HEX_PATH, addr, info['creator']['creationBytecode'])
                time.sleep(0.5)
        if addr not in [filename.lower().split('.')[0] for filename in os.listdir(HEX_PATH)]:
            bytecode = get_bytecode_by_contract_addr(addr, chain)
            save_bytecode_by_contract_addr(HEX_PATH, addr, bytecode)
            time.sleep(0.5)
        if addr not in [filename.lower().split('.')[0] for filename in os.listdir(SOL_PATH)]:
            source = get_source_code_by_contract_addr(addr, chain)
            if 'SourceCode' in source:
                save_sol_by_contract_addr(SOL_PATH, addr, source['SourceCode'])
                time.sleep(0.5)


  0%|          | 0/91 [00:00<?, ?it/s]

Searching 0x02d3aea48b443a0026ed9cbc91b97d7335aba323...
Searching in 1 chain...
Saved 0x02d3aea48b443a0026ed9cbc91b97d7335aba323.json
Saved 0x02d3aea48b443a0026ed9cbc91b97d7335aba323.hex
Saved 0x02d3aea48b443a0026ed9cbc91b97d7335aba323.sol


  1%|          | 1/91 [00:05<08:47,  5.86s/it]

Searching in 97 chain...
Searching in 56 chain...
Searching in 57054 chain...
Searching in 146 chain...
Searching 0x0886949c1b8c412860c4264ceb8083d1365e86cf...
Searching in 1 chain...
error from get_internal_transactions_by_contract_addr: No transactions found
Saved 0x0886949c1b8c412860c4264ceb8083d1365e86cf.json
Saved 0x0886949c1b8c412860c4264ceb8083d1365e86cf.hex
Saved 0x0886949c1b8c412860c4264ceb8083d1365e86cf.sol


  2%|▏         | 2/91 [00:13<10:28,  7.06s/it]

...

Searching in 97 chain...
Searching in 56 chain...
Searching in 57054 chain...
Searching in 146 chain...
Searching 0xee7596c357c50f3677b794a5223db265394fb2e4...
Searching in 1 chain...
error from get_internal_transactions_by_contract_addr: No transactions found
Saved 0xee7596c357c50f3677b794a5223db265394fb2e4.json
Saved 0xee7596c357c50f3677b794a5223db265394fb2e4.hex
Saved 0xee7596c357c50f3677b794a5223db265394fb2e4.sol


 99%|█████████▉| 90/91 [12:30<00:12, 12.64s/it]

Searching in 97 chain...
Searching in 56 chain...
Searching in 57054 chain...
Searching in 146 chain...
Searching 0xfb5fb992b64c86fbcb33ba151abd8ecbcc611bde...
Searching in 1 chain...
error from get_internal_transactions_by_contract_addr: No transactions found
Saved 0xfb5fb992b64c86fbcb33ba151abd8ecbcc611bde.json
Saved 0xfb5fb992b64c86fbcb33ba151abd8ecbcc611bde.hex
Saved 0xfb5fb992b64c86fbcb33ba151abd8ecbcc611bde.sol


100%|██████████| 91/91 [12:37<00:00,  8.33s/it]

Searching in 97 chain...
Searching in 56 chain...
Searching in 57054 chain...
Searching in 146 chain...


In [10]:
len(target_df), len(os.listdir(HEX_PATH)), len(os.listdir(TXN_PATH)), len(os.listdir(SOL_PATH))

(91, 91, 91, 91)

In [11]:
if os.path.exists(target):
    shutil.move(target, DATASET_PATH)
    print("File moved successfully.")
else:
    print("File does not exist.", UNLABELED_PATH)

File moved successfully.
